<a href="https://colab.research.google.com/github/shashigharti/secure-privateai-scholarship-challenge/blob/master/course-exercise-n-projects/Encrypted_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install syft

In [0]:
import string
import torch as th
import syft as sy

In [0]:
char2index = {}
index2char = {}
hook = sy.TorchHook(th)

In [0]:
#create three workers
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [0]:
for i,char in enumerate(' ' + string.ascii_lowercase + '0123456789' + string.punctuation):
  char2index[char] = i
  index2char[i] = char

In [0]:
char2int

In [0]:
int2char

In [0]:
def string2values(str_input, max_len = 8):
  max_len = 8

  str_input = str_input[:max_len].lower()

  if(len(str_input) < max_len):
    str_input = str_input + "." * (max_len - len(str_input))
  values = list()
  for char in str_input:
    values.append(char2int[char])
  return th.tensor(values).long()

def values2string(input_values):
  s = ""
  for value in input_values:
    s += int2char[int(value)]
  return s

In [294]:
string2values("Howdy")

tensor([ 8, 15, 23,  4, 25, 50, 50, 50])

In [0]:
def one_hot(index, length):
  vect = th.zeros(length).long()
  vect[index] = 1
  return vect

In [0]:
def strings_equal(str_a, str_b):
  vect = (str_a * str_b).sum(1)
  x = vect[0]
  for i in range(vect.shape[0] - 1):
    x = x * vect[i + 1]

  return x

In [0]:
def string2one_hot_matrix(str_input, max_len = 8):
  str_input = str_input[:max_len].lower()
  
  #pad strings shorter than max length
  if(len(str_input) < max_len):
    str_input = str_input + "." * (max_len - len(str_input))
    
  char_vectors = list()
  for char in str_input:
    char_v = one_hot(char2int[char], len(int2char)).unsqueeze(0)
    char_vectors.append(char_v)
    
  return th.cat(char_vectors, dim = 0)

In [0]:
keys = list()
values = list()
keys.append(string2one_hot_matrix("key1"))
values.append(string2values("value1"))

keys.append(string2one_hot_matrix("key2"))
keys.append(string2values("value2"))


In [0]:
def query(query_str):
  query_matrix = string2one_hot_matrix(query_str)
  key_matches = list()
  for key in keys:
    key_match = strings_equal(key, query_matrix)
    key_matches.append(key_match)

  result = values[0] * key_matches[0]

  for i in range(len(values) - 1):
    result += values[i+1] * key_matches[i+1]
    
  return values2string(result).replace(".","")

In [0]:
query("key1")

In [0]:
class EncryptedDB:
  
  def __init__(self, *owners, max_key_len = 8, max_val_len = 8):    
    self.max_key_len = 8
    self.max_val_len = 8 
    
    self.keys = list()
    self.values = list()
    self.owners = owners
    
  def add_entry(self, key, value):
    key = string2one_hot_matrix(key)
    key = key.share(*self.owners)
    self.keys.append(key)
    
    value = string2values(value)
    value = value.share(*self.owners)
    self.values.append(value)
    
  def query(self, query_str):    
    query_matrix = string2one_hot_matrix(query_str)
    query_matrix = query_matrix.share(*self.owners)
    
    key_matches = list()
    for key in self.keys:
      
      key_match = strings_equal(key, query_matrix)
      key_matches.append(key_match)

    result = self.values[0] * key_matches[0]

    for i in range(len(self.values) - 1):
      result += self.values[i+1] * key_matches[i+1]

    result =  result.get()
    
    return values2string(result).replace(".","")
  
  

In [0]:
db = EncryptedDB(bob, alice, secure_worker)
db.add_entry("key1", "value1")
db.add_entry("key2", "value2")
db.add_entry("key3", "value3")
db.add_entry("key4", "value4")

In [306]:
db.query("key1")

'value1'